In [142]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotify_config

from IPython.display import IFrame

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from matplotlib import pyplot
import pickle

from fuzzywuzzy import fuzz
from fuzzywuzzy import process


<br>**Authenticating with Spotify**

In [143]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= spotify_config.client_id, client_secret= spotify_config.client_secret))

<br>**Scraping Data from url Billboard Hot 100**

In [144]:
url="https://www.billboard.com/charts/hot-100/"
response = requests.get(url)
print(response.status_code)
soup = BeautifulSoup(response.content, "html.parser")

200


<br>**Extracting Data from Billboard Hot 100**

In [145]:
title=[]
counter=6
while counter<406:
    title.append(str(soup.select("h3#title-of-a-story.c-title")[counter].get_text("/", strip=True)))
    counter = counter + 4
artist = []
counter=0
while counter<100:
    artist.append(str(soup.select("span.c-label.a-no-trucate")[counter].get_text("/", strip=True)))
    counter = counter + 1
hot100_df=pd.DataFrame(list(zip(artist, title)),
              columns=['artist','title'])

<br>**KMeans modeling:**

In [146]:
songs_df = pd.read_excel("all_songs_features.xlsx")
songs_df.drop_duplicates
songs_df.head()
songs_df2=songs_df.drop(columns=["Unnamed: 0","Unnamed: 0.1"])
songs_df2_id = songs_df2.id
songs_df2 = songs_df2.drop('id', axis=1)

In [147]:
scaler = StandardScaler()
scaler.fit(songs_df2)
songs_df2_scaled = scaler.transform(songs_df2)
songs_df2_scaled_df = pd.DataFrame(songs_df2_scaled, columns = songs_df2.columns)
display(songs_df2.head())
print()
display(songs_df2_scaled_df)

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.597,0.620,-8.079,0.0313,0.6110,0.897000,0.1180,0.225,116.029,221047
1,0.642,0.621,-4.343,0.0551,0.2290,0.193000,0.1550,0.364,178.028,216067
2,0.674,0.924,-4.943,0.1020,0.0143,0.000006,0.2160,0.811,123.040,240053
3,0.771,0.594,-6.587,0.0410,0.1220,0.012600,0.0605,0.657,124.025,219462
4,0.883,0.624,-5.820,0.0847,0.2200,0.506000,0.2790,0.402,107.998,171111


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.016521,-0.102502,0.050397,-0.686231,1.161949,3.214452,-0.448430,-1.302116,-0.107332,-0.069260
1,0.256051,-0.098163,0.797133,-0.443176,-0.093914,0.375213,-0.198714,-0.752648,2.004429,-0.129780
2,0.426383,1.216434,0.677208,0.035785,-0.799761,-0.403133,0.212980,1.014347,0.131471,0.161710
3,0.942702,-0.215306,0.348612,-0.587171,-0.445687,-0.352342,-0.836501,0.405584,0.165022,-0.088522
4,1.538864,-0.085147,0.501917,-0.140889,-0.123502,1.637545,0.638171,-0.602433,-0.380877,-0.676108
...,...,...,...,...,...,...,...,...,...,...
8446,-0.632870,-0.176258,0.068386,-0.575937,0.750999,-0.402755,0.435699,1.160609,0.768791,-1.134720
8447,0.740433,0.747865,0.986616,-0.705635,1.563036,-0.402751,-0.272954,1.492661,-0.284382,-0.786988
8448,1.703874,-0.774986,-0.614590,-0.374753,1.355917,-0.403158,0.253474,1.492661,-0.305704,-1.137794
8449,0.240082,0.266279,-0.309180,-0.310415,1.043595,-0.403150,-0.137972,1.405695,1.487107,-1.318417


In [148]:
kmeans = KMeans(n_clusters=15, random_state=1234)
kmeans.fit(songs_df2_scaled_df)

C:\Users\frede\.conda\envs\Ironhack\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=15, random_state=1234)

In [149]:
labels=kmeans.labels_
df_recommender1 = pd.DataFrame({'id': songs_df2_id, 'clusters': labels}, columns=['id', 'clusters'])

In [150]:
df_recommender1.head()

,id,clusters
0,0GAr2dDEqEXqlhQzvyZTsK,9
1,4FXzptXwH5Wz33SVat2uRt,5
2,0yM2YvOGR3cBIoWBeFdtmS,7
3,6Bgsu7Wc90sme8EfplwtxU,7
4,0r5Z0wgmOYDWHnhN4f8DtV,9


In [151]:
df_recommender1.nunique()

id          4331
clusters      15
dtype: int64

<br>**Recommender**

In [152]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [ ]:
x = input("Name a song in the current Billboard Hot 100:")
y = hot100_df['title'].sample().iloc[0]
artist = hot100_df.loc[hot100_df['title'] == y,'artist'].iloc[0]
closest_match = process.extractOne(x, hot100_df['title'])
result=sp.search(q=y+artist,limit=1,market="GB")
list_of_user_audio_features=[]

if x in list(hot100_df["title"]):
        if y!=x:
            print("Great taste! You might also like:", y ,"by", hot100_df.loc[hot100_df['title'] == y,'artist'].iloc[0])
        else:
            c = hot100_df['title'].sample().iloc[0]
            print("Great taste! You might also like:", c ,"by", hot100_df.loc[hot100_df['title'] == c,'artist'].iloc[0])
else:
        print('Sorry, your input is not in the Top 100! Did you mean:',closest_match[0],"by", hot100_df.loc[hot100_df['title'] == closest_match[0],'artist'].iloc[0],"?")
        a = input("Input Y or N:")
        if a in ["Y","y"]:
            print("Great taste! You might also like:", y ,"by", artist)
            song_id=result["tracks"]["items"][0]["id"]
                     
        else:
            result2=sp.search(q=x,limit=1,market="GB")
            song_id = result2["tracks"]["items"][0]["id"]
            print("Did you mean:",result2["tracks"]["items"][0]["name"],"by",result2["tracks"]["items"][0]["artists"][0]["name"],"?")
            b = input("Input Y or N:")
            if b in ["Y","y"]:
                usersong_features = sp.audio_features(result["tracks"]["items"][0]["id"])
                list_of_user_audio_features.append(sp.audio_features(result["tracks"]["items"][0]["id"]))
                user_audio_features_df = pd.DataFrame(list_of_user_audio_features[0])    
                user_audio_features_df =df[["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms"]]
                user_audio_features_scaled = scaler.transform(user_audio_features_df)
                user_audio_features_scaled_df = pd.DataFrame(user_audio_features_scaled, columns = user_audio_features_df.columns)
                k=kmeans.predict(user_audio_features_scaled_df)
                random_row = df_recommender1[df_recommender1['clusters'] == k[0]].sample(1)
                song_id = random_row.iloc[0][0]
                print("You might also like this song:")
                print(k[0])
                            
play_song(song_id)